In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
# 10.2 데이터 집계

In [3]:
# 데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말한다.
# 위 예제에서 mean, count, min, sum을 이용해서 스칼라값을 구했다. 
# GroupBy 객체에 대해 mean()을 수행하면 어떤 일이 생길까?

In [4]:
# [표 10-1]에 있는 것과 같은 많은 일반적인 데이터 집계는 데이터 묶음에 대한 준비된 통계를 계산해내는 최적화된 구현을 갖고 있다.
# 하지만 이 메서드만 사용해야 하는 건 아니다. 표 10-1 페이지 396-397

In [5]:
# 직접 고안한 집계함수를 사용하고 추가적으로 그룹 객체에서 이미 정의된 메서드를 연결해서 사용할 수도 있다.
# 예를 들어 quantile 메서드가 Series나 DataFrame의 컬럼의 변위치를 계산한다는 점을 생각해보자.

In [6]:
# quantile 메서드는 GroupBy만을 위해 구현되지 않았지만 Series 메서드이기 때문에 여기서 사용할 수 있다. 
# 내부적으로 GroupBy는 Series를 효과적으로 잘게 자르고 각 조각에 대해 piece.quantile(0.9)를 호출한다. 
# 그리고 이 결과들을 모두 하나의 객체로 합쳐서 반환한다. 

In [7]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [8]:
grouped = df.groupby("key1")

In [9]:
grouped["data1"].quantile(0.9)

key1
a    1.210513
b    1.713629
Name: data1, dtype: float64

In [10]:
# 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넘기면 된다. 

In [11]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [12]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


In [13]:
# describe 같은 메서드는 데이터를 집계하지 않는데도 잘 작동함을 확인할 수 있다.

In [14]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.555881  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.705025  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.936175  1.393406   3.0  0.441920  0.283299  0.274992  0.278369   
b     1.335403  1.965781   2.0 -0.144516  1.628757 -1.296221 -0.720368   

                                    
           50%       75%       max  
key1                                
a     0.281746  0.525384  0.769023  
b    -0.144516  0.431337  1.007189  

[2 rows x 24 columns]

In [15]:
# 이에 대한 내용은 10.3절에서 더 자세히 다룬다. 

In [16]:
# NOTE_ 사용자 정의 집계함수는 일반적으로 표 10-1에 있는 함수에 비해 무척 느리게 동작하는데, 그 이유는 중간 데이터를 생성하는 과정에서 함수 호출이나 데이터 정렬 같은 오버헤드가 발생하기 때문이다.

In [17]:
# 10.2.1 컬럼에 여러 가지 함수 적용하기

In [18]:
# 앞서 살펴본 팁 데이터로 다시 돌아가자. 여기서는 read_csv 함수로 데이터를 불러온 다음 팁의 비율을 담기 위한 컬럼인 tip_pct를 추가했다.

In [19]:
tips = pd.read_csv("examples/tips.csv")

In [20]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [21]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [22]:
# 이미 살펴봤듯이 Series나 DataFrame의 모든 컬럼을 집계하는 것은 mean이나 std 같은 메서드를 호출하거나 원하는 함수에 aggregate를 사용하는 것이다. 
# 하지만 컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러 개의 함수를 한 번에 적용하기 원한다면 이를 쉽고 간단하게 수행할 수 있다. 
# 앞으로 몇몇 예제를 통해 이를 자세히 알아볼텐데, 먼저 tips를 day와 smoker별로 묶어보자.

In [23]:
grouped = tips.groupby(["day", "smoker"])

In [26]:
# [표 10-1]의 내용과 같은 기술 통계에서는 함수 이름을 문자열로 넘기면 된다. 

In [28]:
grouped_pct = grouped["tip_pct"]

In [29]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [30]:
# 만일 함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다.

In [31]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [32]:
# 여기서는 데이터 그룹에 대해 독립적으로 적용하기 위해 agg에 집계함수들의 리스트를 넘겼다. 

In [33]:
# GroupBy 객체에서 자동으로 지정하는 컬럼 이름을 그대로 쓰지 않아도 된다. 
# lambda 함수는 이름(함수 이름은 __name__ 속성으로 확인 가능하다)이 "<lambda>"인데, 이를 그대로 쓸 경우 알아보기 힘들어진다.

In [34]:
# 이때 이름과 함수가 담긴 (name, function) 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소가 DataFrame에서 컬럼 이름으로 사용된다(2개의 튜플을 가지는 리스트가 순서대로 맵핑된다).

In [35]:
grouped_pct.agg([("foo", "mean"), ("bar", np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [36]:
# DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다.
# tip_pct와 total_bill 컬럼에 대해 동일한 세 가지 통계를 계산한다고 가정하자. 

In [37]:
functions = ["count", "mean", "max"]

In [39]:
result = grouped["tip_pct", "total_bill"].agg(functions)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_34292\3417596921.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped["tip_pct", "total_bill"].agg(functions)


In [40]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [41]:
# 위에서 확인할 수 있듯이 반환된 DataFrame은 계층적인 컬럼을 가지고 있으며 이는 각 컬럼을 따로 계산한 다음 concat 메서드를 이용해서 keys 인자로 컬럼 이름을 넘겨서 이어붙인 것과 동일하다.

In [42]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [43]:
# 위에서처럼 컬럼 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다. 

In [44]:
ftuples = [("Durchschnitt", "mean"), ("Abweichung", np.var)]

In [45]:
grouped["tip_pct", "total_bill"].agg(ftuples)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_34292\547607895.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped["tip_pct", "total_bill"].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [46]:
# 컬럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어 있는 사전을 넘기면 된다.

In [47]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [50]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [51]:
# 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적인 컬럼을 가지게 된다.

In [52]:
# 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

In [53]:
# 지금까지 살펴본 모든 예제에서 집계된 데이터는 유일한 그룹키 조합으로 색인(어떤 경우에는 계층적 색인)되어 반환되었다. 
# 하지만 항상 이런 동작을 기대하는 것은 아니므로 groupby 메서드에 as_index=False를 넘겨서 색인되지 않도록 할 수 있다. 

In [54]:
tips.groupby(["day", "smoker"], as_index=False).mean()

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_34292\3417973839.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(["day", "smoker"], as_index=False).mean()


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [55]:
# 물론 이렇게 하지 않고 색인된 결과에 대해 reset_index 메서드를 호출해서 같은 결과를 얻을 수 있다. as_index=False 옵션을 사용하면 불필요한 계산을 피할 수 있다.